<a href="https://colab.research.google.com/github/jiztastamablastamarang/TwitterSturm/blob/main/TwitterSturm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###CTRL + F9 Щоб запустити весь код, або Runtime => Run all###


##Credentials##
Реєструємо API Twitter та створюємо додаток з можливістю читати та писати повідомлення. Після цього буде згенеровано подібні ключі доступу

In [6]:
KEY = ""
SECRET = ""
TOKEN = ""
TOKEN_SECRET = ""

##Docs##
Гуглотаблиці списком користувачей, контентом, директорія з фото

In [36]:
# USERS = "https://docs.google.com/spreadsheets/d/1uUF6WGtFl-WBPe_oLdk4gdgFp09aPOAekBWmHdIUnC0/edit#gid=0" #повний список сенаторів
USERS = "https://docs.google.com/spreadsheets/d/1UgpJjBFaGD89dYYsxJLJQn1GNyd9Fo6Rg1df4XSNnuQ/edit#gid=0" #обрізані сенатори
CONTENT = "https://docs.google.com/spreadsheets/d/1dsdTocyIgQpJYrBLnjxBl9e3Bdw65vAWmpBPorzLlrI/edit#gid=0"# коменти
IMGS = "https://drive.google.com/drive/folders/1V7YZjkovjvvJAoTBEpAGp4VwlRWdneXA" # директорія з фото
# IMGS = "https://docs.google.com/spreadsheets/d/1TmQXycbTh7EsgArWT2mqUfQRPXBmidxZB5lsmyFfnRQ/edit#gid=0" #або URL з фото

In [37]:
def load_doc(url):
  return pd.read_csv(url.replace("/edit#gid=", "/export?format=csv&gid="), on_bad_lines="skip")

##Modules##

In [9]:
import tweepy
import pandas as pd
import requests
import os
import glob
import random
import shutil
import time

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [10]:
!pip install -U -q PyDrive

##API##

In [11]:
def swad():
  auth = tweepy.OAuthHandler(KEY, SECRET)
  auth.set_access_token(TOKEN, TOKEN_SECRET)
  api = tweepy.API(auth, 
                   wait_on_rate_limit=True, 
                   wait_on_rate_limit_notify=True)
  try:
      api.verify_credentials()
  except:
      print("ups!")
  return api
  
api = swad()

##Images##

In [12]:
# images = load_doc(IMGS)

# def load_img(url):
#   file_name = "new.jpg"
#   request = requests.get(url, stream=True)
#   if request.status_code==200:
#     with open(file_name, "wb") as file:
#       for chunk in request:
#         file.write(chunk)
#       media = api.media_upload(file_name)
#       os.remove(file_name)
#       return media
#   else:
#     print("ups!")

# imgs = []
# for url in images["URL"]:
#     imgs.append(load_img(url))

Тут може вискочити повідомлення про підтвердження дій с гуглодиском. Треба погодитися і ввести відповідний код, нажати enter

In [13]:
folder_id = IMGS.split("/")[-1]
imgs = []

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

path = "./temp"
try:
    os.makedirs(path)
except: pass

file_list = drive.ListFile(
    {'q': "'{}' in parents".format(folder_id)}).GetList()

for image in file_list:
    fname = os.path.join(path, image["title"])
    new_file = drive.CreateFile({"id": image["id"]})
    new_file.GetContentFile(fname)
    print("uploading: " + image["title"])
    imgs.append(api.media_upload(os.path.join("/content", "temp", image["title"] )))
shutil.rmtree(path)

uploading: photo_2022-02-25_07-17-45.jpg
uploading: photo_2022-02-25_07-17-44.jpg
uploading: 27_02_22_Bucha_fire3.jfif
uploading: 27.02.22_Троєщина.jpeg
uploading: 2022-02-27_Житомир_обстріл.jpg
uploading: Бородянка_Після обстрілів.jpg
uploading: gettyimages-1238718491.jpg
uploading: 275186063_3244844625791550_7558080127054411714_n.jpg
uploading: mariupol.jpeg
uploading: 274660241_3059356657712999_6548491878968273213_n.jpg
uploading: mariupol-2.jpeg
uploading: pid-novoju-kahovkoju-okupanti-rozstrilyali-mashinu-miscevih-zhiteliv.jpeg
uploading: original.jpg
uploading: gettyimages-1238853230.jpg
uploading: 274483441_3271747659721513_1428952723540980161_n.jpg
uploading: Бородянка після обстрілів.jpg
uploading: Харків2.jpg
uploading: Kharkivsky Avenue, Kyiv Andriy Dubchak (Donbas Frontliner).jpg
uploading: 228485_Harkiv_bombarduvanna_960x380_0.jpg
uploading: 19_main.jpg
uploading: Cz-8c_NPtMRVO2gwxO_1TVWBl7Osi-SA.jpg
uploading: FMg41ReXwAQgTa7.jpg
uploading: 24ukraine-briefing-morning-kyiv

##Content##

In [14]:
content = load_doc(CONTENT)
content

,Messedge,Lengh
0,"We are fighting against armies and tanks, we d...",184
1,Russia is deliberately and continuously destro...,228
2,Ukraine brush with brand new naZis under the f...,244
3,Don't let Russia hurt our people. We call to #...,192


In [15]:
# Довжина твіта не повинна перевищувати 280 знаків
messeges = [m for m in content["Messedge"].to_list() if len(m)<=280]

##Test tweet##

In [16]:
def add_comment(user, status, media):
    #останній твіт цільового користувача, який будемо коментувати
    last_tweet = api.user_timeline(screen_name=user, count=1, include_rts=False, tweet_mode="extended")[0]

    #створюємо повідомлення та додаємо фото
    api.update_status(
                      status=status, 
                      in_reply_to_status_id=last_tweet.id, 
                      media_ids=[media]
    )

In [17]:
user = "jiztastamablast" # тестуємо на собі. пишемо свій нік
status= "!!!" #  комент
media = random.choice(imgs).media_id # рандомне фото

In [18]:
api.update_status(status="!", media_ids=[random.choice(imgs).media_id]) 

Status(_api=<tweepy.api.API object at 0x7f694b759a10>, _json={'created_at': 'Tue Mar 08 14:22:06 +0000 2022', 'id': 1501201597896048640, 'id_str': '1501201597896048640', 'text': '! https://t.co/5jeyOtwsKg', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1501201562487734285, 'id_str': '1501201562487734285', 'indices': [2, 25], 'media_url': 'http://pbs.twimg.com/media/FNVW3gxXwA0F93r.jpg', 'media_url_https': 'https://pbs.twimg.com/media/FNVW3gxXwA0F93r.jpg', 'url': 'https://t.co/5jeyOtwsKg', 'display_url': 'pic.twitter.com/5jeyOtwsKg', 'expanded_url': 'https://twitter.com/jiztastamablast/status/1501201597896048640/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 1125, 'h': 2000, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 383, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 675, 'h': 1200, 'resize': 'fit'}}}]}, 'extended_entities': {'media': [{'id': 1501201562487734285, 'id_str': '1501

In [19]:
add_comment(user, status, media)

##Users##

In [39]:
users = load_doc(USERS)
try:
  users["TwitterNickname"] = users["Twitter"].str.replace("https://twitter.com/", "", regex=False).str.replace("@", "", regex=False)
except:
  pass
#прибираємо нульові строки
users.dropna(subset=["TwitterNickname"], inplace=True)
users

,State,Member of Congress,Name,Party,Twitter,TwitterNickname
0,California 12th District,U.S. Representative,"Pelosi, Nancy",D,@SpeakerPelosi,SpeakerPelosi
1,California 23rd District,U.S. Representative,"McCarthy, Kevin",R,@GOPLeader,GOPLeader
2,New York 5th District,U.S. Representative,"Meeks, Gregory W.",D,@RepGregoryMeeks,RepGregoryMeeks
3,New Jersey 4th District,U.S. Representative,"Smith, Chris",R,@RepChrisSmith,RepChrisSmith
4,Arizona 4th District,U.S. Representative,"Gosar, Paul A.",R,@RepGosar,RepGosar
...,...,...,...,...,...,...
85,Utah,U.S. Senator,"Lee, Mike",R,@SenMikeLee,SenMikeLee
86,Utah,U.S. Senator,"Romney, Mitt",R,@SenatorRomney,SenatorRomney
87,West Virginia,U.S. Senator,"Manchin, Joe",D,@Sen_JoeManchin,Sen_JoeManchin
88,Wisconsin,U.S. Senator,"Johnson, Ron",R,@SenRonJohnson,SenRonJohnson


In [ ]:
# users.info()

##Sturm##

In [40]:
bad_user = []

for user in users["TwitterNickname"].to_list():
  try:
    # обираємо рандомні повідомлення та фото
    status= random.choice(messeges)
    media = random.choice(imgs).media_id

    # додаємо комент
    add_comment(user, status, media)

  except tweepy.TweepError as error:
    if error.api_code == 185:
      print("Перевищено ліміт 300 повідомельнь за 3 години. Чекаємо...")
      time.sleep(3*60*60)
    else:
      print(error)
      print(f"error sending comment to user: {user}")

    bad_user.append(user)


Not authorized.
error sending comment to user: RepChrisSmith
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
error sending comment to user: RepThomasMassi
Not authorized.
error sending comment to user: mtgreenee
[{'code': 187, 'message': 'Status is a duplicate.'}]
error sending comment to user: JoaquinCastrotx


In [41]:
bad_user

['RepChrisSmith', 'RepThomasMassi', 'mtgreenee', 'JoaquinCastrotx']